## Part A: Initiate the model ingredients and define functions (to be used in the optimization step).

Load modules and packages:

In [136]:
using Random, Distributions, Optim, NLsolve # JuMP, Ipopt

### Step 1: Set the parameter values.

In [137]:
N = 3 # Number of countries
J = 250000 # Number of goods (needs to be big number and an integer type)
theta = 4 # Frechet shape parameter (governs comparative advantage)
T = ones(N, 1) * 1.5 # Frechet scale parameter (governs absolute advantage)
sigma = 2 # Substitution elasticity between goods
tau = ones(N, N) # Iceberg trade costs
L = ones(N, 1) # Size of labor force in each country

Random.seed!(1) # Seed for the random number generator (to guarantee reproducibility; this is standard in research these days)

TaskLocalRNG()

### Step 2 : Draw a $J$-by-$N$ matrix of uniform random variables on $[0,1]$ and convert them to Fréchet random variables using the inverse CDF.

These are the productivities for each good in each country.

Note: In the 'Distributions' package, the inverse CDF can be calculated using the 'quantile()' function for any distribution defined in the package, of which 'Frechet' is one. More on the 'Distributions' package is available at https://juliastats.org/Distributions.jl/v0.14/index.html.

Since you will be comparing economies with different shape parameters ($\theta$), you can write a simple function where $\theta$ is an argument to transform the uniform draws into Fréchet-distributed productivities.

Save the productivities in an array. You can call it $z$.

Let $n \in {1,2,3}$ index the three countries and WLOG let $n=1$ be the "Home" country. Normalize the wage in "Home" to $w = 1$. The wages in countries 2 and 3 are endogenous variables.

In [138]:
# Draw the J x N uniform r.v.:
u = rand(J, N)
# Define inverse Fréchet CDF:
function inv_Frechet(u, T, theta)
    return quantile.(Frechet(theta, T[1]), u)
end
# Initiate array of productivities:
z = Array{Float64}(undef, J, N)
# Evaluate the inv_Frechet function country-by-country:
for n in 1:N
    z[:, n] = inv_Frechet(u[:, n], T[n], theta)
end
# Initialize the 1 x (N-1) vector of wages:
wage = Array{Float64}(undef, 1, N-1)
#  The wage in country 1 is given by 'w1' and we normalize it to unity:
w1 = 1

1

### Step 3: Create a function that computes the trades shares for all countries, given the productivities and wages.

Calculating these shares involves a couple of substeps:
1. identify the lowest-cost producer of good $j$ in each of the $N$ different countries
1. compute the trade shares given the selection in the previous sub-step.

Note 1: Even though the wages in countries 2 and 3 are endogenous, you need to use a place holder for the purposes of defining the function. The function accepts these wages as arguments and in the optimization step the wages will be such that trade is balanced at the global scale.

Note 2: We use 'w' to denote the vector of $N-1$ wages used as arguments in the functions below; 'wage' is an array that stores the equilibrium wages computed in Part B. Recall that the wage in country 1 is normalized to unity and we denote it 'w1'; it is a separate input argument for all functions and it is treated like a parameter (which is useful in anticipation of the optimization step below).

In [139]:
# In this economy, goods indexed by the same j in [1,J] in all countries are perfect substitutes for one another. Countries select the lowest cost provider for each one of these J different goods.

# Initialize the arrays for the returns:
P = Array{Float64}(undef, N, 1)
pi = Array{Float64}(undef, N, N)

# Define a function to identify lowest-cost producer in each country for all J products across all other N countries (including the domestically produced good:
function lcp(tau,w1,w,z,T,theta,sigma,N)
    for n in 1:N
        w_tmp = [w1 w[:]']
        # In country 'n', find lowest cost producer for each j in {1,...,J}:
        p_all = tau[n, :]' .* w_tmp ./ z
        i = argmin(p_all, dims=2)
        # Compute local prices in country 'n':
        p_n = vec(p_all[i])
        # Compute the price index in country 'n':
        P[n] = only((ones(1, J) * (p_n .^ (1 - sigma))./J) .^ (1 / (1 - sigma))) # 'only' selects a single element from a collection; in this case, it converts the type from a 1 x 1 vector to a Float64 scalar 
        # Compute the expenditure shares:
        Phi_n = only(ones(1, N) * (T .* (w_tmp' .* tau[n, :]) .^ (-theta)))
        for m in 1:N
            pi[n, m] = (T[m] .* (w_tmp[m] .* tau[n, m])^(-theta)) ./ Phi_n
        end
    end
    return P, pi
end

lcp (generic function with 2 methods)

### Step 4: Create a function that computes the trade balance for each country, again for given productivities and wages.

Note: Use the shares and country-specific incomes to compute bilateral trade balances and then sum them across countries.

In [140]:
function trade_balance(tau,w1,w,z,T,theta,sigma,N,L)
    # Compute the price indices and trade shares using the 'lcp' function defined above:
    P,pi = lcp(tau,w1,w,z,T,theta,sigma,N)
    # Compute total income for each country:
    inc = [w1 w[:]'] .* L
    tb = Array{Float64}(undef,1,N)
    for n in 1:N
        tb[1,n] = 0
        for i in 1:N
            tb[1,n] = tb[1,n] + pi[n,i] .* inc[i] - pi[i,n] .* inc[n]
        end
    end
    return tb[2:N]
end

trade_balance (generic function with 2 methods)

### Step 5: Create a function that reports some additional equilibrium results.

The functions you have defined so far will be used in a non-linear optimizer that solves the system of equations. Let the output of this optimization be 'sol' and treat it as a placeholder for now. In the optimization step 'sol' will be the vector of wages in all N countries.

In particular, store the following results in arrays:
1. equilibrium wages, and
2. welfare (i.e., $\tfrac{wL}{P}$).

You can also print these results to the screen using the 'println' function or command.

In [141]:
function report_results(w1, sol, L, tau, z, T, theta, sigma, N)
    income =  [w1 sol[:]'].* L
    tb = trade_balance(tau, w1, sol[:]', z, T, theta, sigma, N, L)
    P, pi = lcp(tau, w1, sol, z, T, theta, sigma, N)
    welfare = [w1 sol[:]'] ./ P[:]'
    # Print results to screen:
    println("Equilibrium wages:\n", round.(sol[:]',digits=3))
    println("Trade shares:\n", round.(pi,digits=3))
    println("Trade balances in countries 2 and 3:\n", round.(tb,digits=3))
    println("Welfare (w/P):\n", round.(welfare, digits=3))
end

report_results (generic function with 2 methods)

Note 1: This particular function will not be called by the non-linear optimizer and, therefore, there is no need to write it in a way such that additional parameters (other than the wage) can be passed to the solver.

Note 2: 'sol' is a placeholder array for the output arguments of the non-linear solver (i.e., wages in country 2,...,N).

# Part B: Solve the model.

### Question a.
Countries are symmetric in terms of size and productivities. Trade is free (i.e., the iceberg costs of international trade between any pair of origin and destination are normalized to 1). Symmetry implies that wages are equalized across countries, provided that J is large enough and therefore that law of large numbers applies. In other words, we don't actually have to solve for the wage vector and we could instead compute the trade shares for a given vector of unit-value wages. Also, since international trade is free, the expenditure shares $\pi$ calculated using the 'lcp' function are symmetric as well (namely, $\pi$ is an $N \times N$ matrix with identical elements equal to $\tfrac{1}{N}$)

There are several options for solving systems of non-linear equations numerically. In this particular case, we are looking for the root of the sum of country-specific trade balances (in other words, the vector of wages such that the *global* trade balance equals zero). In other work, I have used the *JuMP.jl* package to solve systems of equations. The package has advantages and drawbacks. The advantage is that it can call a number of different solvers and it can handle a variety of optimization problems (constrained, unconstrained, linear, non-linear,...). The drawback is that the syntax has to fit the requirements of the package and it can be "clunky" to work with.

For this problem, we can use the *Optim.jl* package, which is a solver with a more standard syntax (similar to solvers commonly used in *MATLAB*, for instance). Chapter 9 of the *julia.quantecon.org* website has some helpful material and references to get you started with the *Optim* package.

There are two options for the optimization step in this problem.

1. Use the _Optim_ package and minimize the squared trade balances.
1. Use the _NLsolve_ package and find the zeros of the trade balance function.

Here, I will use the _Optim_ package first since I pointed you in that direction. I will also provide the solution for the root-finding problem based on _NLsolve_.

In [142]:
# Since we are minimizing the squared trade balances, we need an auxiliary function that sums the squares of the N-1 trade balances for countries 2,...,N.
function trade_balance_sq(tau,w1,w,z,T,theta,sigma,N,L)
    tbsq = sum(trade_balance(tau,w1,w,z,T,theta,sigma,N,L).^2)
    return tbsq
end

# Recall that we are solving for the wages in countries 2,...,N such that trade is balanced in these N-1 countries. When trade is indeed balance, then Walras Law immediately implies that the trade is balanced in country 1. This guarantees that the number of unknowns equals the number of equations in the system that we are solving.

# Our objective function 'trade_balance_sq' accepts several input arguments, only one of which is 'w'. To "feed" the objective function to the optimizer, we need to create an anonymous function that redefines 'trade_balance_sq' as a function of a single argument 'w'.
sol_a = optimize(w -> trade_balance_sq(tau,w1,w,z,T,theta,sigma,N,L), [1.1 1.1])
# Now we use the minimizer from the optimization step and "feed" it to the 'report_results' function as an input argument:
report_results(w1, Optim.minimizer(sol_a), L, tau, z, T, theta, sigma, N)

# Next, let's find the solution by way of a root-finding algorithm instead, which is available in the 'NLsolve' package.
root_a = nlsolve(w -> trade_balance(tau,w1,w,z,T,theta,sigma,N,L), [1.1 1.1])
# Results based on root-finding algorithm:
report_results(w1, root_a.zero, L, tau, z, T, theta, sigma, N)


Equilibrium wages:
[1.0 1.0]
Trade shares:
[0.333 0.333 0.333; 0.333 0.333 0.333; 0.333 0.333 0.333]
Trade balances in countries 2 and 3:
[0.0, 0.0]
Welfare (w/P):
[2.417 2.417 2.417]
Equilibrium wages:
[1.0 1.0]
Trade shares:
[0.333 0.333 0.333; 0.333 0.333 0.333; 0.333 0.333 0.333]
Trade balances in countries 2 and 3:
[0.0, 0.0]
Welfare (w/P):
[2.417 2.417 2.417]


### Question b.

Countries remain symmetric but international trade is subject to uniform iceberg costs $\tau = 1.1$.


In [143]:
# Update the matrix of iceberg costs:
tau = tau .* 1.1
# Make an adjustment for the fact that domestic trade is costless:
for n in 1:N
    tau[n,n]=1
end
# Use the result from question a. as the initial guess for the optimization in question b.:
sol_b = optimize(w -> trade_balance_sq(tau,w1,w,z,T,theta,sigma,N,L), Optim.minimizer(sol_a))
# Now we use the minimizer from the optimization step and "feed" it to the 'report_results' function as an input argument:
report_results(w1, Optim.minimizer(sol_b), L, tau, z, T, theta, sigma, N)
# Find the solution by way of a root-finding algorithm instead, which is available in the 'NLsolve' package.
root_b = nlsolve(w -> trade_balance(tau,w1,w,z,T,theta,sigma,N,L), root_a.zero)
# Results based on root-finding algorithm:
report_results(w1, root_b.zero, L, tau, z, T, theta, sigma, N)


Equilibrium wages:
[1.0 1.0]
Trade shares:
[0.423 0.289 0.289; 0.289 0.423 0.289; 0.289 0.289 0.423]
Trade balances in countries 2 and 3:
[-0.0, 0.0]
Welfare (w/P):
[2.278 2.278 2.278]
Equilibrium wages:
[1.0 1.0]
Trade shares:
[0.423 0.289 0.289; 0.289 0.423 0.289; 0.289 0.289 0.423]
Trade balances in countries 2 and 3:
[0.0, 0.0]
Welfare (w/P):
[2.278 2.278 2.278]


### Question c.
Countries are still symmetric but country 3 is "remote", i.e. $\tau_{13} = \tau_{31} = \tau_{23} = \tau_{32} = 1.3$. The other trade costs are the same as in b.

In [144]:
tau[1,3] = 1.3
tau[3,1] = tau[1,3]
tau[2,3] = 1.3
tau[3,2] = tau[2,3]

# Use the result from question a. as the initial guess for the optimization in question c.:
sol_c = optimize(w -> trade_balance_sq(tau,w1,w,z,T,theta,sigma,N,L), Optim.minimizer(sol_a))
# Now we use the minimizer from the optimization step and "feed" it to the 'report_results' function as an input argument:
report_results(w1, Optim.minimizer(sol_c), L, tau, z, T, theta, sigma, N)
# Find the solution by way of a root-finding algorithm instead, which is available in the 'NLsolve' package.
root_c = nlsolve(w -> trade_balance(tau,w1,w,z,T,theta,sigma,N,L), root_a.zero)
# Results based on root-finding algorithm:
report_results(w1, root_c.zero, L, tau, z, T, theta, sigma, N)


Equilibrium wages:
[1.0 0.969]
Trade shares:
[0.481 0.328 0.191; 0.328 0.481 0.191; 0.191 0.191 0.618]
Trade balances in countries 2 and 3:
[0.0, -0.0]
Welfare (w/P):
[2.205 2.206 2.071]
Equilibrium wages:
[1.0 0.969]
Trade shares:
[0.481 0.328 0.191; 0.328 0.481 0.191; 0.191 0.191 0.618]
Trade balances in countries 2 and 3:
[0.0, -0.0]
Welfare (w/P):
[2.205 2.206 2.071]


### Question d.
Country 2 is more productive than 1 and 3. Everything else is as in c.

In [145]:
# Increase T in country 2 from its original value to 2:
T[2] = 2
# Update the productivities in country 2 accordingly:
z[:, 2] = inv_Frechet(u[:, 2], T[2], theta)

# Use the result from question c. as the initial guess for the optimization in question d.:
sol_d = optimize(w -> trade_balance_sq(tau,w1,w,z,T,theta,sigma,N,L), Optim.minimizer(sol_c))
# Now we use the minimizer from the optimization step and "feed" it to the 'report_results' function as an input argument:
report_results(w1, Optim.minimizer(sol_d), L, tau, z, T, theta, sigma, N)
# Find the solution by way of a root-finding algorithm instead, which is available in the 'NLsolve' package.
root_d = nlsolve(w -> trade_balance(tau,w1,w,z,T,theta,sigma,N,L), root_c.zero)
# Results based on root-finding algorithm:
report_results(w1, root_d.zero, L, tau, z, T, theta, sigma, N)


Equilibrium wages:
[1.075 0.969]
Trade shares:
[0.481 0.328 0.191; 0.328 0.481 0.191; 0.191 0.191 0.618]
Trade balances in countries 2 and 3:
[-0.0, -0.0]
Welfare (w/P):
[2.42 2.69 2.195]
Equilibrium wages:
[1.075 0.969]
Trade shares:
[0.481 0.328 0.191; 0.328 0.481 0.191; 0.191 0.191 0.618]
Trade balances in countries 2 and 3:
[-0.0, 0.0]
Welfare (w/P):
[2.42 2.69 2.195]


### Question e.
Change the value of the shape parameter in the Fréchet distribution but keep everything else as in c.

In [146]:
theta = 8
T[2] = T[1]
# Update the productivities in all N countries:
for n in 1:N
    z[:, n] = inv_Frechet(u[:, n], T[n], theta)
end

# Use the result from question c. as the initial guess for the optimization in question d.:
sol_e = optimize(w -> trade_balance_sq(tau,w1,w,z,T,theta,sigma,N,L), Optim.minimizer(sol_c))
# Now we use the minimizer from the optimization step and "feed" it to the 'report_results' function as an input argument:
report_results(w1, Optim.minimizer(sol_e), L, tau, z, T, theta, sigma, N)
# Find the solution by way of a root-finding algorithm instead, which is available in the 'NLsolve' package.
root_e = nlsolve(w -> trade_balance(tau,w1,w,z,T,theta,sigma,N,L), root_c.zero)
# Results based on root-finding algorithm:
report_results(w1, root_e.zero, L, tau, z, T, theta, sigma, N)


Equilibrium wages:
[1.0 0.983]
Trade shares:
[0.622 0.29 0.088; 0.29 0.622 0.088; 0.088 0.088 0.824]
Trade balances in countries 2 and 3:
[0.0, -0.0]
Welfare (w/P):
[1.734 1.734 1.674]
Equilibrium wages:
[1.0 0.983]
Trade shares:
[0.622 0.29 0.088; 0.29 0.622 0.088; 0.088 0.088 0.824]
Trade balances in countries 2 and 3:
[0.0, -0.0]
Welfare (w/P):
[1.734 1.734 1.674]
